In [1]:
import pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta
# 판다스 엑셀 익스포트를 위한 라이브러리
import openpyxl


PROJECT = "ballosodeuk"
bq = bigquery.Client(project=PROJECT)

### 원본 데이터 로드.
- 25년 이전 가입자 쿼리로 컷.
- 첫구매 기준, 파이썬으로 최소 한달 전으로 최신 구매 컷 (2025-03-13 데이터 컷)
    - 결과적으로 2025-03-13 ~ 2025-04-25 데이터 컷

In [2]:
query = """
  WITH first_purchase AS (
    SELECT 
      user_id,
      MIN(register_dt) AS first_purchase_date
    FROM `dm.agg_user_cash_daily`
    WHERE purchase IS NOT NULL
    GROUP BY user_id
  ),

  second_purchase AS (
    SELECT 
      a.user_id,
      MIN(a.register_dt) AS second_purchase_date
    FROM `dm.agg_user_cash_daily` a
    JOIN first_purchase fp ON a.user_id = fp.user_id
    WHERE a.purchase IS NOT NULL
      AND a.register_dt > fp.first_purchase_date
    GROUP BY a.user_id
  ),

  final AS (
    SELECT DISTINCT 
      a.user_id,
      date_diff (current_date(), join_dt, day) as join_dt,
      b.register_dt AS p_date,
      b.current_shoji AS cur_shoji,
      b.current_cash AS cur_cash,
      b.pre_shoji AS pre_shoji,
      b.pre_cash AS pre_cash,
      b.purchase AS p_amt,
      b.spend_use - b.earn_recall - b.spend_burnt AS s_amt,
      ROUND(
        (b.spend_use - b.earn_recall - b.spend_burnt) / NULLIF(b.purchase, 0)
      , 2) AS s_rto,
      SUM(a.earn_exchange) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS earn_exchange,
      COUNT(a.earn_exchange) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS earn_exchange_cnt,
      SUM(a.earn_reward) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS earn_reward,
      COUNT(a.earn_reward) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS since_visit_dt,
      COUNT(CASE WHEN a.earn_reward > 0 THEN a.earn_reward END) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS earn_reward_cnt
    FROM `dm.agg_user_cash_daily` a
    JOIN first_purchase fp ON a.user_id = fp.user_id
    JOIN `dm.agg_user_cash_daily` b 
      ON a.user_id = b.user_id 
      AND b.register_dt = fp.first_purchase_date
    JOIN `dw.dim_airbridge_member` c 
      ON a.user_id = c.user_id
    WHERE a.register_dt <= fp.first_purchase_date
      AND b.purchase IS NOT NULL
      AND c.join_dt < "2024-12-31"
  ),

  second_purchase_data AS (
    SELECT DISTINCT 
      a.user_id,
      s.second_purchase_date,
      b.current_shoji AS second_cur_shoji,
      b.pre_shoji AS second_pre_shoji,
      b.pre_cash AS second_pre_cash,
      b.purchase AS second_p_amt,
      b.spend_use - b.earn_recall - b.spend_burnt AS second_s_amt,
      ROUND(
        (b.spend_use - b.earn_recall - b.spend_burnt) / NULLIF(b.purchase, 0)
      , 2) AS second_s_rto,
      SUM(a.earn_exchange) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS second_earn_exchange,
      COUNT(a.earn_exchange) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS second_earn_exchange_cnt,
      SUM(a.earn_reward) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS second_earn_reward,
      COUNT(a.earn_reward) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS second_since_visit_dt,
      COUNT(CASE WHEN a.earn_reward > 0 THEN a.earn_reward END) OVER (
        PARTITION BY a.user_id
        ORDER BY a.register_dt
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS second_earn_reward_cnt
    FROM `dm.agg_user_cash_daily` a
    JOIN second_purchase s ON a.user_id = s.user_id
    JOIN first_purchase fp ON a.user_id = fp.user_id
    JOIN `dm.agg_user_cash_daily` b 
      ON a.user_id = b.user_id 
      AND b.register_dt = s.second_purchase_date
    WHERE a.register_dt > fp.first_purchase_date
      AND a.register_dt <= s.second_purchase_date
      AND b.purchase IS NOT NULL
  ),

  between_purchases AS (
    SELECT
      a.user_id,
      SUM(a.earn_exchange) AS between_earn_exchange,
      COUNT(a.earn_exchange) AS between_earn_exchange_cnt,
      SUM(a.earn_reward) AS between_earn_reward,
      COUNT(CASE WHEN a.earn_reward > 0 THEN a.earn_reward END) AS between_earn_reward_cnt,
      COUNT(a.register_dt) AS between_visit_cnt
    FROM `dm.agg_user_cash_daily` a
    JOIN first_purchase fp ON a.user_id = fp.user_id
    JOIN second_purchase sp ON a.user_id = sp.user_id
    WHERE a.register_dt > fp.first_purchase_date
      AND a.register_dt < sp.second_purchase_date
    GROUP BY a.user_id
  ),

  churn AS (
    SELECT 
      DISTINCT user_id, 
      is_churn
    FROM `dm.agg_user_churn_daily`
    WHERE is_churn IS TRUE
  )


  SELECT 
    a.*,
    CASE WHEN b.is_churn IS TRUE THEN TRUE ELSE FALSE END AS is_churn,
    s.second_purchase_date,
    s.second_cur_shoji,
    s.second_pre_shoji,
    s.second_pre_cash,
    s.second_p_amt,
    s.second_s_amt,
    s.second_s_rto,
    s.second_earn_exchange,
    s.second_earn_exchange_cnt,
    s.second_earn_reward,
    s.second_since_visit_dt,
    s.second_earn_reward_cnt,
    bp.between_earn_exchange,
    bp.between_earn_exchange_cnt,
    bp.between_earn_reward,
    bp.between_earn_reward_cnt,
    date_diff(s.second_purchase_date, a.p_date, day) AS days_between_purchases
  FROM (
    SELECT *
    FROM (
      SELECT *, ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY earn_reward_cnt DESC) AS rnk
      FROM final
    )
    WHERE rnk = 1
  ) a
  LEFT JOIN churn b ON a.user_id = b.user_id
  LEFT JOIN (
    select *
    from(
        SELECT *, ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY second_earn_reward_cnt DESC) AS rnk
        FROM second_purchase_data
      )
    where rnk = 1
  ) s ON a.user_id = s.user_id
  LEFT JOIN between_purchases bp ON a.user_id = bp.user_id
"""

In [ ]:
df = bq.query(query).to_dataframe()

In [33]:
from datetime import datetime as dt
ts = dt.today().strftime('%Y-%m-%d')
df.to_parquet(f'df_{ts}.parquet')


In [3]:
df = pd.read_parquet('C:\FTC_downloads\code\git\project_hct\/04-25\df_2025-04-17.parquet')

In [4]:
# 12/31 ~ 3/13 데이터 컷 (변수 삭제)
df['p_date'] = pd.to_datetime(df['p_date'])
df = df[(df['p_date'] > '2024-12-30') & (df['p_date'] <= '2025-03-13')]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
import pandas as pd

# 한글 폰트 설정
font_path = "C:/Windows/Fonts/malgun.ttf"  # 윈도우의 경우 맑은 고딕 폰트 경로
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# 기존 히스토그램 그리기
plt.figure(figsize=(15, 5))
df.pre_shoji.hist(bins=100)
plt.xlabel('현재 쇼지 금액', fontsize=12)
plt.tight_layout()

## 소지 쇼지 기반 코호트 분석

In [ ]:
# pre_shoji를 2000 단위로 구간화하고, 3만 이상은 하나의 구간으로 통합
bins = list(range(0, 30001, 2000)) + [float('inf')]
labels = [f'{i}-{i+1999}' for i in range(0, 30000, 2000)] + ['30000+']

# 구간화 및 각 구간별 유저 수 계산
df['pre_shoji_bin'] = pd.cut(df['pre_shoji'], bins=bins, labels=labels, right=False)
bin_counts = df['pre_shoji_bin'].value_counts().sort_index()

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 첫 번째 그래프: 구간별 유저 수 막대 그래프
bin_counts.plot(kind='bar', ax=axes[0])
axes[0].set_title('쇼지 금액 구간별 유저 수', fontsize=14)
axes[0].set_xlabel('쇼지 금액 구간', fontsize=12)
axes[0].set_ylabel('유저 수', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)

# 두 번째 그래프: 구간별 유저 비율 파이 차트
bin_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
axes[1].set_title('쇼지 금액 구간별 유저 비율', fontsize=14)
axes[1].set_ylabel('')

plt.tight_layout()


In [ ]:
df['p_date'] = pd.to_datetime(df.p_date)
df.query("p_date >= '2025-01-01'").groupby('pre_shoji_bin').agg({
    'user_id': 'count',
    'pre_shoji': 'mean',
    'p_amt': 'mean',
    's_amt': 'mean',
    's_rto': 'mean',
    'earn_exchange': 'mean',
    'earn_reward': 'mean',
    # 'earn_reward_cnt': 'mean',
    'earn_reward_cnt': 'median',
    })


In [ ]:
df.query("p_date >= '2025-01-01'").earn_reward_cnt.hist()

In [9]:
# BigQuery에서 가져온 데이터는 때때로 타입 문제가 발생할 수 있습니다.
# 'earn_reward_cnt' 열의 타입을 명시적으로 변환해 보겠습니다.
df['earn_reward_cnt'] = df['earn_reward_cnt'].astype(float)

# 참고: df = bq.query(query).to_dataframe()으로 BigQuery에서 데이터를 가져올 때
# 'unknown type object' 오류가 발생할 수 있습니다. 이는 pandas와 BigQuery 간의
# 데이터 타입 호환성 문제로 인한 것입니다.

In [10]:
df['pre_cash'] = df['pre_cash'].fillna(0)
# 5000 단위로 구간 설정
bins = list(range(0, 30001, 5000)) + [float('inf')]
labels = [f'{i}-{i+4999}' for i in range(0, 30000, 5000)] + ['30000+']

# pre_cash와 pre_shoji 모두 5000 단위로 구간화하고, 3만 이상은 하나의 구간으로 통합
df['pre_cash_bin'] = pd.cut(df['pre_cash'], bins=bins, labels=labels, right=False)
df['pre_shoji_bin'] = pd.cut(df['pre_shoji'], bins=bins, labels=labels, right=False)

In [ ]:
df.groupby('pre_cash_bin').agg({
    'user_id': 'count',
    'is_churn': 'count',
    'pre_cash': 'mean',
    'pre_shoji': 'mean',
    'p_amt': 'mean',
    's_amt': 'mean',
    's_rto': 'mean',
    'earn_exchange': 'mean',
    'earn_reward': 'mean',
    'earn_reward_cnt': ['median','mean'],
    })


In [ ]:
df.groupby('pre_shoji_bin').agg({
    'user_id': 'count',
    'is_churn': 'count',
    'pre_cash': 'mean',
    'pre_shoji': 'mean',
    'p_amt': 'mean',
    's_amt': 'mean',
    's_rto': 'mean',
    'earn_exchange': 'mean',
    'earn_reward': 'mean',
    'earn_reward_cnt': ['median','mean'],
    })


In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# 메모리 사용량 확인
import psutil
import os

# 현재 프로세스의 메모리 사용량 확인
process = psutil.Process(os.getpid())
memory_info = process.memory_info()
print(f"메모리 사용량: {memory_info.rss / 1024 / 1024:.2f} MB")

# 시스템 전체 메모리 상태 확인
system_memory = psutil.virtual_memory()
print(f"시스템 전체 메모리: {system_memory.total / 1024 / 1024 / 1024:.2f} GB")
print(f"사용 가능한 메모리: {system_memory.available / 1024 / 1024 / 1024:.2f} GB")
print(f"메모리 사용률: {system_memory.percent}%")

# CPU 사용량 확인
cpu_percent = psutil.cpu_percent(interval=1)
print(f"CPU 사용률: {cpu_percent}%")

# 각 CPU 코어별 사용량
cpu_percent_per_core = psutil.cpu_percent(interval=1, percpu=True)
for i, percent in enumerate(cpu_percent_per_core):
    print(f"CPU 코어 {i}: {percent}%")

In [ ]:
df.sample(1)

In [ ]:
df['is_first_bankrupt'] = df.cur_shoji < 5
df[['is_first_bankrupt','cur_shoji','p_amt','s_amt']].sample(10)

In [ ]:
df.pre_cash_bin.unique()

In [ ]:
df.groupby(['pre_shoji_bin','pre_cash_bin']).agg({
    'join_dt': ['median','mean','min','max'],
    'user_id': 'count',
    'is_first_bankrupt': 'sum',
    'is_churn': 'sum',
    "second_cur_shoji" :'count',
    'pre_cash': 'mean',
    'pre_shoji': 'mean',
    'p_amt': 'mean',
    's_amt': 'mean',
    's_rto': 'mean',
    'earn_exchange': 'mean',
    'earn_reward': 'mean',
    'earn_reward_cnt': ['median','mean','max'],
    }).reset_index()


In [ ]:
df.columns

In [ ]:
df.columns

In [ ]:
df['pre_asset'] = df['pre_cash'] + df['pre_shoji']
df['second_pre_asset'] = df['second_pre_cash'] + df['second_pre_shoji']

df.groupby("earn_reward_cnt").agg({
    "user_id":'count',
    "pre_shoji":'mean',
    "pre_cash":"mean",
    "is_first_bankrupt":'sum',
    "is_churn":'sum',
    "second_cur_shoji":'count',
    "earn_reward":'mean',
    "earn_exchange":'mean',
    "p_amt":'mean',
    "s_amt":'mean',
    "s_rto":'mean',
    'second_pre_shoji':'mean',
    'second_pre_cash':'mean',
    'between_earn_reward_cnt':'mean',
    'between_earn_reward':'mean',
    'pre_asset':'mean',
    'second_pre_asset':'mean'
    }).reset_index()


#### 리워드 축적, 첫 구매까지 삼각분포를 이룸. 바로 사용은 이탈하는 특수 케이스로 간주

In [ ]:
# 리워드 획득 횟수별 사용자 분포 분석
distribution_df = df.groupby("earn_reward_cnt")["user_id"].count().reset_index()
distribution_df = distribution_df.sort_values("earn_reward_cnt")

# 시각화 개선
plt.figure(figsize=(15, 6))
plt.bar(distribution_df["earn_reward_cnt"], distribution_df["user_id"])
plt.title("리워드 획득 횟수별 사용자 분포", fontsize=14)
plt.xlabel("리워드 획득 횟수", fontsize=12)
plt.ylabel("사용자 수", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# 기본 통계량 출력
print("리워드 획득 횟수 통계:")
print(f"평균: {df['earn_reward_cnt'].mean():.2f}")
print(f"중앙값: {df['earn_reward_cnt'].median():.2f}")
print(f"최소값: {df['earn_reward_cnt'].min():.2f}")
print(f"최대값: {df['earn_reward_cnt'].max():.2f}")

In [ ]:
# 첫 번째 누적 금액 계산
df["first_cum_money"] = df.earn_reward + df.earn_exchange

# 5천 단위로 구간화
df["first_cum_money_bin"] = pd.cut(df["first_cum_money"], 
                                  bins=range(0, df["first_cum_money"].max().astype(int) + 5001, 5000),
                                  labels=[f"{i}-{i+4999}" for i in range(0, df["first_cum_money"].max().astype(int) + 5000, 5000)][:-1],
                                  right=False)

# 구간별 집계
df.groupby("first_cum_money_bin").agg({
    "user_id":'count',
    "pre_cash":'mean',
    "cur_shoji":'mean',
    "is_first_bankrupt":'sum',
    "is_churn":'sum',
    "second_cur_shoji":'count',
    # "earn_reward":'mean',
    "earn_exchange":'mean',
    "p_amt":'mean',
    "s_amt":'mean',
    "s_rto":'mean',
    }).reset_index()


In [ ]:
df.columns

### 재구매 통계

In [ ]:
# 첫 번째 누적 금액 계산
df["first_cum_money"] = df.earn_reward + df.earn_exchange

# 5천 단위로 구간화
df["first_cum_money_bin"] = pd.cut(df["first_cum_money"], 
                                  bins=range(0, df["first_cum_money"].max().astype(int) + 5001, 5000),
                                  labels=[f"{i}-{i+4999}" for i in range(0, df["first_cum_money"].max().astype(int) + 5000, 5000)][:-1],
                                  right=False)

# 구간별 집계
df.groupby("first_cum_money_bin").agg({
    "user_id":'count',
    # "pre_cash":'mean',
    "cur_shoji":'mean',
    "is_first_bankrupt":'sum',
    "is_churn":'sum',
    # "earn_reward":'mean',
    "earn_exchange":'mean',
    "p_amt":'mean',
    "s_amt":'mean',
    "s_rto":'mean',
    "second_cur_shoji":'count',
    "earn_reward_cnt":'mean',
    "second_earn_reward":'mean',
    "second_earn_exchange":'mean',
    "second_earn_reward_cnt":'mean',
    "second_p_amt":'mean',
    "second_s_amt":'mean',
    "second_s_rto":'mean',
    }).reset_index()


In [ ]:
df.columns

In [ ]:
# float 타입 오류 수정
# NaN 값을 제외하고 최대값 계산
max_value = df["second_p_amt"].dropna().max()
if pd.notna(max_value):
    max_int = int(max_value) + 5001
    # 레이블 개수는 구간 경계 개수보다 하나 적어야 함
    bins = list(range(0, max_int, 5000))
    labels = [f"{i}-{i+4999}" for i in range(0, max_int, 5000) if i < max_int - 5000]
    df["second_p_amt_bin"] = pd.cut(df["second_p_amt"], 
                                    bins=bins,
                                    labels=labels,
                                    right=False)
else:
    # 데이터가 없는 경우 빈 카테고리 생성
    df["second_p_amt_bin"] = pd.Categorical([])

df.groupby("second_p_amt_bin").agg({
    "user_id":'count',
    "second_pre_shoji":'mean',
    "second_pre_cash":'mean',
    "second_p_amt":'mean',
    "second_s_amt":'mean',
    "second_s_rto":'mean',
    "second_earn_reward":'mean',
    "second_earn_exchange":'mean',
    "second_earn_reward_cnt":'mean',
    }).reset_index()

In [ ]:
df.columns

In [ ]:
# 첫구매 시 쇼지 and 재산 5000미만 유저 필터링
tg = df.query("pre_shoji_bin == '0-4999' & pre_cash_bin == '0-4999'")[[
    'user_id','pre_shoji','pre_cash', 'cur_shoji','cur_cash',  'p_amt','s_amt','s_rto']]

tg['shoji_spent'] = tg['pre_shoji'] - tg['cur_shoji']
tg['cash_spent'] = tg['pre_cash'] - tg['cur_cash']


tg.user_id.nunique()


## 수익성 분석

### 첫구매 시 재산 적었던 유저 or 전체유저 필터링하여 분석 
- 첫 구매시 쇼지 <= 5000 and 캐시가 <= 5000인 유저 필터링
- 해당 타겟의 전체 구매 평균 통계값 도출하여 분석

In [ ]:
# 리스트의 요소들을 쉼표로 구분된 문자열로 변환
user_ids_str = ", ".join([f"'{user_id}'" for user_id in tg.user_id.tolist()])

# 저재산 유저
tg_query_1 = """
    SELECT 
        user_id, register_dt, pre_shoji, pre_cash, current_shoji, current_cash, purchase, earn_total, spend_total, 
        spend_use, earn_reward,earn_cash_direct_exchange, earn_exchange, earn_recall
    FROM dm.agg_user_cash_daily
    WHERE 1=1 
    and user_id IN ({})
        and purchase is not null
        and register_dt > "2024-12-31"
""".format(user_ids_str)
# 전체 유저
tg_query_2 = """
    SELECT 
        user_id, register_dt, pre_shoji, pre_cash, current_shoji, current_cash, purchase, earn_total, spend_total, 
        spend_use, earn_reward,earn_cash_direct_exchange, earn_exchange, earn_recall
    FROM dm.agg_user_cash_daily
    WHERE 1=1 
        and purchase is not null
        and register_dt > "2024-12-31"
""".format(user_ids_str)

tg_df = bq.query(tg_query_2).to_dataframe()

tg_df.head()

In [ ]:
tg_df.iloc[:,2:] = tg_df.iloc[:,2:].astype(float)
tg_df['earn_exchange'] = tg_df['earn_exchange'] + tg_df['earn_cash_direct_exchange']
tg_df['s_rto'] = round(
    (tg_df['spend_total']  / tg_df['purchase']).where(
        tg_df['earn_recall'] > tg_df['spend_use'],
        (tg_df['spend_use'] - tg_df['earn_recall']) / tg_df['purchase']
    ), 2)
# cash_rto 계산
# cash_rto 계산 - inf 처리 및 최대값 1 제한
tg_df['cash_rto'] = (tg_df['earn_exchange'] / tg_df['spend_total']).where(
    tg_df['earn_recall'] > tg_df['spend_use'],
    tg_df['earn_exchange'] / (tg_df['spend_use'] - tg_df['earn_recall'])
).clip(upper=1.0).fillna(1.0).round(2)
tg_df.groupby('user_id').agg({
    'purchase':'count',
    'earn_total':'sum',
    'spend_use':'sum',
    'earn_reward':'sum',
    'earn_exchange':'sum',
    's_rto':'mean',
    'cash_rto':'mean',
    }).reset_index().head()

tg_df['cash_direct_rto'] = (tg_df['earn_cash_direct_exchange'] / tg_df['spend_total']).clip(upper=1.0)

tg_df[(tg_df['s_rto'] <= 0.06) & (tg_df['s_rto'] >= 0.04)].cash_direct_rto.mean()
tg_df['s_rto'] = tg_df['s_rto'].clip(lower=0.0)
tg_df['s_rto'] = tg_df['s_rto'].clip(upper=1)

In [ ]:
tg_df.groupby(s_rto_bins)['cash_direct_rto'].mean()

In [ ]:
# s_rto를 50개 구간으로 나누고 각 구간별 cash_direct_rto 평균과 pre_cash 평균 계산
fig, ax1 = plt.subplots(figsize=(10, 6))

# s_rto를 50개 구간으로 나누기
s_rto_bins = pd.cut(tg_df.s_rto, bins=20)

# 각 구간별 cash_direct_rto 평균 계산
s_rto_cash_direct_avg = tg_df.groupby(s_rto_bins)['cash_direct_rto'].mean()

# 각 구간별 pre_cash 평균 계산
s_rto_pre_cash_avg = tg_df.groupby(s_rto_bins)['pre_cash'].mean()

# 첫 번째 y축에 cash_direct_rto 평균 그래프 그리기
s_rto_cash_direct_avg.plot(kind='bar', ax=ax1, color='skyblue', edgecolor='black')
ax1.set_xlabel('쇼핑지원금 사용 비율(s_rto) 구간')
ax1.set_ylabel('캐시 직접 교환 비율(cash_direct_rto) 평균', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# 두 번째 y축 생성하여 pre_cash 평균 라인 그래프 추가
ax2 = ax1.twinx()
s_rto_pre_cash_avg.plot(kind='line', ax=ax2, color='red', linestyle='-', linewidth=1)
ax2.set_ylabel('보유 캐시(pre_cash) 평균', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('(총 구매자)\n'
          '1. 쇼지 사용 비율 분포 & 쇼지 사용 시 캐시 직접 교환 사용 비율\n'
          '2. 각 구간 별 소지한 캐시')
# plt.title('(첫구매 시 쇼지+캐시 5000미만)\n'
#           '1. 쇼지 사용 비율 분포 & 쇼지 사용 시 캐시 직접 교환 사용 비율\n'
#           '2. 각 구간 별 소지한 캐시')
# plt.text(0.5, 0.95, '5% 사용 구간에 캐시 직접 교환 비율이 높은 경향', 
#          ha='center', va='center',style='italic', transform=plt.gca().transAxes, fontsize=12)
plt.xticks(rotation=90)
ax1.grid(True, alpha=0.3)
plt.tight_layout()


In [ ]:
# s_rto와 cash_direct_rto 관계 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

# s_rto 히스토그램
ax1.hist(tg_df.s_rto, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('쇼핑지원금 사용 비율(s_rto)')
ax1.set_ylabel('유저 수')
ax1.set_title('쇼지 사용 비율의 분포')
ax1.grid(True, alpha=0.3)

# s_rto가 10% 미만인 구간에서의 cash_direct_rto 분석
low_s_rto_users = tg_df[tg_df.s_rto < 0.1]
s_rto_bins_low = pd.cut(low_s_rto_users.s_rto, bins=10)
cash_direct_by_s_rto = low_s_rto_users.groupby(s_rto_bins_low)['cash_direct_rto'].mean()

# 전체 구간 대비 각 구간의 cash_direct_rto 비중을 파이 차트로 표현
total_cash_direct = low_s_rto_users['cash_direct_rto'].sum()
cash_direct_proportion = low_s_rto_users.groupby(s_rto_bins_low)['cash_direct_rto'].sum() / total_cash_direct

# 파이 차트 그리기
# 상위 3개 미만 기타로 통합
sorted_proportions = cash_direct_proportion.sort_values(ascending=False)
top_3 = sorted_proportions.iloc[:3]
others = pd.Series({'기타': sorted_proportions.iloc[3:].sum()})
combined_proportions = pd.concat([top_3, others])

combined_proportions.plot(kind='pie', ax=ax2, autopct='%1.1f%%', startangle=90, 
                         colors=plt.cm.Paired(np.linspace(0, 1, len(combined_proportions))),
                         wedgeprops={'edgecolor': 'black', 'linewidth': 1})
ax2.set_ylabel('')  # y 레이블 제거
ax2.set_title('쇼핑지원금 사용 비율 10% 미만 구간별\n캐시 직접 교환 비율 비중')

plt.suptitle('(총 구매자)\n'
             '쇼핑지원금 사용 비율 분포 & 낮은 구간의 캐시 직접 사용 집중도', fontsize=15)
# plt.suptitle('(첫구매 시 쇼지+캐시 5000미만)\n'
#              '쇼핑지원금 사용 비율 분포 & 낮은 구간의 캐시 직접 사용 집중도', fontsize=15)
plt.tight_layout()

In [ ]:
# 전체 
pd.cut(tg_df.s_rto, bins=20).value_counts()

In [ ]:
# s_rto 비율에 대한 히스토그램 생성
plt.figure(figsize=(10, 6))
plt.hist(tg_df.cash_rto, bins=20, edgecolor='black')
plt.title('소비 비율(s_rto) 분포')
plt.xlabel('소비 비율')
plt.ylabel('유저 수')
plt.title('구매 시 쇼핑지원금 내 캐시 교환액 비율')
plt.grid(True, alpha=0.3)

# 각 비율 구간별 유저 수 계산
cash_rto_counts = tg_df.cash_rto.value_counts(bins=10).sort_index()
print("소비 비율 구간별 유저 수:")
print(cash_rto_counts)

In [ ]:
# cash_rto 비율과 purchase 평균을 히스토그램으로 표현
fig, ax1 = plt.figure(figsize=(10, 6)), plt.gca()

# cash_rto 히스토그램
n, bins, patches = ax1.hist(tg_df.query("cash_rto < 0.99").cash_rto, bins=20, edgecolor='black', alpha=0.7, label='유저 수')

# 두 번째 y축 생성
ax2 = ax1.twinx()

# 각 구간별 purchase 평균 계산
purchase_means = []
for i in range(len(bins)-1):
    mask = (tg_df.cash_rto >= bins[i]) & (tg_df.cash_rto < bins[i+1]) & (tg_df.cash_rto < 0.99)
    purchase_means.append(tg_df[mask].purchase.mean())

# purchase 평균 선 그래프 추가
bin_centers = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
ax2.plot(bin_centers, purchase_means, color='red', marker='o', linestyle='-', linewidth=2, label='구매 평균')

# 그래프 설정
ax1.set_xlabel('캐시 교환 비율')
ax1.set_ylabel('유저 수')
ax2.set_ylabel('구매 평균', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('첫구매 시 쇼지 5000미만 & 캐시 교환 비율 99% 미만 : 유저의 분포 및 구매 평균')
fig.legend(loc='upper right')
plt.grid(True, alpha=0.3)

# 각 비율 구간별 유저 수 계산
cash_rto_counts = tg_df.cash_rto.value_counts(bins=10).sort_index()
print("캐시 교환 비율 구간별 유저 수:")
print(cash_rto_counts)

In [ ]:
# cash_rto 비율과 purchase 평균을 히스토그램으로 표현
fig, ax1 = plt.figure(figsize=(10, 6)), plt.gca()

# cash_rto 히스토그램
n, bins, patches = ax1.hist(tg_df.cash_rto, bins=20, edgecolor='black', alpha=0.7, label='유저 수')

# 두 번째 y축 생성
ax2 = ax1.twinx()

# 각 구간별 purchase 평균 계산
purchase_means = []
for i in range(len(bins)-1):
    mask = (tg_df.cash_rto >= bins[i]) & (tg_df.cash_rto < bins[i+1])
    purchase_means.append(tg_df[mask].purchase.mean())

# purchase 평균 선 그래프 추가
bin_centers = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
ax2.plot(bin_centers, purchase_means, color='red', marker='o', linestyle='-', linewidth=2, label='구매 평균')

# 그래프 설정
ax1.set_xlabel('캐시 교환 비율')
ax1.set_ylabel('유저 수')
ax2.set_ylabel('구매 평균', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('첫구매 시 쇼지 5000미만 & 캐시 교환 비율 99% 미만 : 유저의 분포 및 구매 평균')
fig.legend(loc='upper right')
plt.grid(True, alpha=0.3)

# 각 비율 구간별 유저 수 계산
cash_rto_counts = tg_df.cash_rto.value_counts(bins=10).sort_index()
print("캐시 교환 비율 구간별 유저 수:")
print(cash_rto_counts)

In [ ]:
# s_rto 비율과 purchase 평균을 히스토그램으로 표현
fig, ax1 = plt.figure(figsize=(12, 6)), plt.gca()

# s_rto 히스토그램
n, bins, patches = ax1.hist(tg_df.s_rto, bins=30, edgecolor='black', alpha=0.7, label='유저 수')
plt.xticks(rotation=45)
ax1.set_xticks(np.arange(0, 1.1, 0.05))  # x축 눈금을 0부터 1까지 0.05 간격으로 설정

# 두 번째 y축 생성
ax2 = ax1.twinx()

# 각 구간별 purchase 평균 계산
purchase_means = []
for i in range(len(bins)-1):
    mask = (tg_df.s_rto >= bins[i]) & (tg_df.s_rto < bins[i+1])
    purchase_means.append(tg_df[mask].purchase.mean())
purchase_medians = []
for i in range(len(bins)-1):
    mask = (tg_df.s_rto >= bins[i]) & (tg_df.s_rto < bins[i+1])
    purchase_medians.append(tg_df[mask].purchase.median())

# purchase 평균 선 그래프 추가
bin_centers = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
ax2.plot(bin_centers, purchase_means, color='red', marker='o', linestyle='-', linewidth=2, label='구매 평균')
ax2.plot(bin_centers, purchase_medians, color='blue', marker='o', linestyle='-', linewidth=2, label='구매 중간값')

### 쇼지 사용액 당 bep 그래프 추가
import numpy as np
shoji_purchase_means = np.multiply(purchase_means, bins[:len(purchase_means)])
shoji_price_cut = np.multiply(shoji_purchase_means, 7.143)
shoji_price_cut = np.minimum(np.maximum(shoji_price_cut, 2600), 26000)
# 광고 O
shoji_purchase_means_ad = np.multiply(purchase_means, bins[:len(purchase_means)])
shoji_price_cut_ad = np.multiply(shoji_purchase_means_ad, 5.0)
shoji_price_cut_ad = np.minimum(np.maximum(shoji_price_cut_ad, 2600), 26000)
# ax2 축에 쇼지 사용액 당 bep 그래프 추가
ax2.plot(bin_centers, shoji_price_cut, color='green', marker='x', linestyle='--', linewidth=1, label='BEP : 광고X')
ax2.plot(bin_centers, shoji_price_cut_ad, color='brown', marker='x', linestyle='--', linewidth=1, label='BEP : 광고O')

# purchase_means < shoji_price_cut 조건을 만족하는 최댓값 찾기
valid_indices = [i for i in range(len(purchase_means)) if purchase_means[i] > shoji_price_cut[i]]
if valid_indices:
    max_index = max(valid_indices)
    # 해당 위치에 수직선 추가
    ax2.axvline(x=bin_centers[max_index], color='green', linestyle='--', linewidth=1.5, 
                label=f'수익 마지노선: {bin_centers[max_index]:.2f}')
valid_indices_ad = [i for i in range(len(purchase_means)) if purchase_means[i] > shoji_price_cut_ad[i]]
if valid_indices_ad:
    max_index = max(valid_indices_ad)
    # 해당 위치에 수직선 추가
    ax2.axvline(x=bin_centers[max_index], color='brown', linestyle='--', linewidth=1.5, 
                label=f'수익 마지노선: {bin_centers[max_index]:.2f}')

# 그래프 설정
ax1.set_xlabel('쇼지사용 비율')
ax1.set_ylabel('유저 수')
ax2.set_ylabel('구매 평균', color='red')
ax2.tick_params(axis='y', labelcolor='black')

# plt.title('(첫 구매 시 쇼지+캐시 5000미만)\n'
        #   ' 유저의 구매 시 쇼지 사용 비율 & 구매 평균 & BEP', fontsize=15)
plt.title('(총 구매자)\n'
          ' 유저의 구매 시 쇼지 사용 비율 & 구매 평균 & BEP', fontsize=15)
fig.legend(loc='upper right')
plt.grid(True, alpha=0.3)

# 각 비율 구간별 유저 수 계산
s_rto_counts = tg_df.s_rto.value_counts(bins=10).sort_index()
print("쇼지사용 비율 구간별 유저 수:")
print(s_rto_counts) 

In [ ]:
[i for i in range(len(purchase_means)) if purchase_means[i] > shoji_price_cut[i]]

### 12/31 이래로, 총 재산이 별로 없을떄 유저들의 구매 패턴 

In [ ]:
## 구매당시 5000원 미만 재산 유저들들
tg_query = """
    with raw as
    (select user_id, register_dt, current_shoji, current_cash, purchase, earn_total, spend_total, spend_use, earn_reward,earn_cash_direct_exchange, earn_exchange, earn_recall, pre_shoji + pre_cash as pre_asset
    from `dm.agg_user_cash_daily`
    where 1=1
    and register_dt > "2024-12-31"
    and purchase is not null)

    select *
    from raw
    where pre_asset <= 2000
"""
tg_df_2 = bq.query(tg_query).to_dataframe()

tg_df_2.head()

In [ ]:
tg_df_2.iloc[:,2:] = tg_df_2.iloc[:,2:].astype(float)
tg_df_2['earn_exchange'] = tg_df_2['earn_exchange'] + tg_df_2['earn_cash_direct_exchange']
tg_df_2['s_rto'] = round(
    (tg_df_2['spend_total']  / tg_df_2['purchase']).where(
        tg_df_2['earn_recall'] > tg_df_2['spend_use'],
        (tg_df_2['spend_use'] - tg_df_2['earn_recall']) / tg_df_2['purchase']
    ), 2)
# cash_rto 계산
# cash_rto 계산 - inf 처리 및 최대값 1 제한
tg_df_2['cash_rto'] = (tg_df_2['earn_exchange'] / tg_df_2['spend_total']).where(
    tg_df_2['earn_recall'] > tg_df_2['spend_use'],
    tg_df_2['earn_exchange'] / (tg_df_2['spend_use'] - tg_df_2['earn_recall'])
).clip(upper=1.0).fillna(1.0).round(2)
tg_df_2.groupby('user_id').agg({
    'purchase':'count',
    'earn_total':'sum',
    'spend_use':'sum',
    'earn_reward':'sum',
    'earn_exchange':'sum',
    's_rto':'mean',
    'cash_rto':'mean',
    }).reset_index().head()


In [ ]:
# s_rto 비율과 purchase 평균을 히스토그램으로 표현
fig, ax1 = plt.figure(figsize=(10, 6)), plt.gca()

# s_rto 히스토그램
n, bins, patches = ax1.hist(tg_df_2.s_rto, bins=20, edgecolor='black', alpha=0.7, label='유저 수')

# 두 번째 y축 생성
ax2 = ax1.twinx()

# 각 구간별 purchase 평균 계산
purchase_means = []
for i in range(len(bins)-1):
    mask = (tg_df_2.s_rto >= bins[i]) & (tg_df_2.s_rto < bins[i+1])
    purchase_means.append(tg_df_2[mask].purchase.mean())

# purchase 평균 선 그래프 추가
bin_centers = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
ax2.plot(bin_centers, purchase_means, color='red', marker='o', linestyle='-', linewidth=2, label='구매 평균')

# 그래프 설정
ax1.set_xlabel('쇼지사용 비율')
ax1.set_ylabel('유저 수')
ax2.set_ylabel('구매 평균', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('12/31 이래, 구매 당시 총 재산이 5000원 미만인 유저들의 구매 패턴 : \n유저의 분포 및 구매 평균')
fig.legend(loc='upper right')
plt.grid(True, alpha=0.3)

# 각 비율 구간별 유저 수 계산
s_rto_counts = tg_df_2.s_rto.value_counts(bins=20).sort_index()
print("쇼지사용 비율 구간별 유저 수:")
print(s_rto_counts)